## Lectura del fichero de estado de vacunación publicado por Sanidad en  
* https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm


In [ ]:
# importamos pandas, numpy y datetime
# previamente ha sido necesatio instalar el motor odfpy con un pip3 install odfpy
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

In [ ]:
pd.__version__

preparamos nombre del fichero csv, dentro del directorio csv. Preparamos dos ficheros:
  * estado_vacunacion_.csv , última versión del mismo con el acumulado de todos los informes publicados
  * estado_vacunacion_year-month-day.csv, con la versión del día en cuestión, solo con la información de ese día

In [ ]:
path_directorio='csv/'
nombre_root_fichero = 'estado_vacunacion_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=1,day=7)
fecha_informe_str=fecha_informe.strftime('%Y%m%d')
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

Enlace al fichero en formato ods

In [ ]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

Leemos el fichero

In [ ]:
try:
    datos_row = pd.read_excel(enlace)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

In [ ]:
datos_row

..copia para tratarlo

In [ ]:
datos_hoy = datos_row.copy()

renombramos columnas, añadimos columna con `fecha_de_informe` y ordenamos

In [ ]:
datos_hoy.columns=['ccaa','Dosis entregadas','Dosis administradas','% sobre entregadas','Fecha de la ultima vacuna registrada']

In [ ]:
datos_hoy['date_pub']=fecha_informe
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace

In [ ]:
datos_hoy=datos_hoy[['date_pub','ccaa','Fecha de la ultima vacuna registrada','Dosis entregadas','Dosis administradas','% sobre entregadas','source_name','source']]

eliminamos la fila Totales

In [ ]:
datos_hoy = datos_hoy.drop(datos_hoy.index[-1])

In [ ]:
datos_hoy

guardamos el fichero de hoy en el directorio csv

In [ ]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora bajamos el acumulado actual para añadirle los datos de hoy, y volverlo a subir 

In [ ]:
datos_acumulados = pd.read_csv('csv/Libro1.csv',sep=';')

In [ ]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [ ]:
datos_acumulados

concatenamos...

In [ ]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [ ]:
#aseguramos formato fecha para evitar inconsistencia en el merge
datos_acumulados['date_pub'] = pd.to_datetime(datos_acumulados['date_pub'],format='%d/%m/%y')
datos_acumulados['Fecha de la ultima vacuna registrada'] = pd.to_datetime(datos_acumulados['Fecha de la ultima vacuna registrada'],format='%d/%m/%y')

In [ ]:
datos_acumulados['% sobre entregadas']=datos_acumulados['Dosis administradas']/datos_acumulados['Dosis entregadas']

In [ ]:
datos_acumulados

y volvemos a subirlo..

In [ ]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)